In [1]:
import sys, os
sys.path.append('../MultiFishLSS/')
from headers import *
from twoPoint import *
from twoPointNoise import *
from classy import Class


## Generate BAO forecasts

#### Fiducial cosmology

In [2]:
default_cosmo = {
        'output': 'tCl lCl mPk',
        'non linear':'halofit',
        'l_max_scalars': 4000,
        'lensing': 'yes',
        'P_k_max_h/Mpc': 2.,
        'z_pk': '0.0,1087',
        'A_s': 2.10732e-9,
        'n_s': 0.96824,
        'alpha_s': 0.,
        'h': 0.6736,
        'N_ur': 2.0328,
        'N_ncdm': 1,
        'm_ncdm': 0.06,
        'tau_reio': 0.0544,
        'omega_b': 0.02237,
        'omega_cdm': 0.1200,
        'Omega_k': 0.,
        'Omega_Lambda': 0.,
        'w0_fld':-1,
        'wa_fld':0}
cosmo = Class()
cosmo.set(default_cosmo)
cosmo.compute()
h = cosmo.h()

chi = lambda zz: cosmo.comoving_distance(zz)*h

#### Experiment configuration

In [3]:
fsky = 5000/41253 # 5000 deg2

# two redshift bins from z = 2.7 to 3.3
zmin, zmax = 2.7, 3.3
nbins = 2

# 2 galaxy samples 
# name, bias, number density as lists
samples=['ga','gb']
b = [lambda z: 2.5, lambda z: 3.5] 
n = [lambda z: 2e-4, lambda z: 2e-4]

# overlap between stochastic terms
# index 0 and 2 are auto-correlations, so they must be 1
# index 1 is the cross-correlation between the two samples, set to 0 by default for no overlap
fover=[1,0,1]

# experiment name and basedir
bd='./'
surveyname='example'

exp = experiment(zmin=zmin, zmax=zmax, nbins=nbins, fsky=fsky, b=b, n=n,samples=samples,fover=fover)
forecast = fisherForecast(experiment=exp,cosmo=cosmo,name=surveyname,basedir=bd,ell=np.arange(10,3000,1))    

#### Compute derivatives (if not done already)

Derivatives are computed faster with dedicated node, using `example_setup.sh`

Turn `forecast.recon` to `True` to work on BAOs (reconstructed power spectrum)

In [4]:
basis = np.array(['alpha_perp','alpha_parallel','b'])

# set recon = True, so that we perform BAO reconstruction when computing the power spectrum
forecast.recon = True

forecast.free_params = basis
forecast.compute_derivatives() # will not overwrite unless specified

#### Use derivatives to construct Fisher matrix and get forecasts

In [5]:
derivs = forecast.load_derivatives(basis)
F = lambda i: forecast.gen_fisher(basis, 100, derivatives=derivs, zbins=np.array([i]))
Fs = [F(i) for i in range(nbins)]
Fs = np.array(Fs)

In [10]:
Finvs = [np.linalg.inv(Fs[i]) for i in range(nbins)]
aperp_err = [np.sqrt(Finvs[i][0,0]) for i in range(nbins)]
aparr_err = [np.sqrt(Finvs[i][1,1]) for i in range(nbins)]

print('Redshifts:',forecast.experiment.zcenters)
print('Relative error on DA/rd:',[f'{_:.5f}' for _ in aperp_err])
print('Relative error on H*rd:',[f'{_:.5f}' for _ in aparr_err])


Redshifts: [2.85 3.15]
Relative error on DA/rd: ['0.00782', '0.00833']
Relative error on H*rd: ['0.01151', '0.01205']


In [ ]:
# Compute aiso and a_ap errors
aiso_err=np.zeros(nbins)
aani_err=np.zeros(nbins) 
for i in range(nbins) :
    # start from marginalized covariance on ap at Finv
    cov  = Finvs[i][0:2,0:2]
    icov = np.linalg.inv(cov)

    J  = np.array([[1,-1/3],[1.,2/3]]) #  d(at,ap) / d(aiso,aani)
    icov_bis = J.T.dot(icov.dot(J))
    cov_bis  = np.linalg.inv(icov_bis)
    aiso_err[i] = np.sqrt(cov_bis[0,0])
    aani_err[i] = np.sqrt(cov_bis[1,1])

print('Redshifts:',forecast.experiment.zcenters)
print('Relative error on DV/rd:',[f'{_:.5f}' for _ in aiso_err])
print('Relative error on DM/DH:',[f'{_:.5f}' for _ in aani_err])

Redshifts: [2.85 3.15]
Relative error on DV/rd: ['0.00514', '0.00546']
Relative error on DM/DH: ['0.01623', '0.01708']


In [8]:
forecast.recon = False
